In [1]:
import model as M
import evaluate as E
import config as CFG
import os
import torch

In [2]:
DATASETS = ["billsum", "cnn_dailymail", "big_patent", "scientific_papers"]

In [3]:
def train(hierarchical, loss_func, suffix='', separate=False, use_pooler=True, use_lscore=False):
    for DATASET in DATASETS:
        train_set = M.CustomDataset(os.path.join(CFG.DATASET_ROOT, DATASET, CFG.METHOD, 'train.tsv'), hierarchical=hierarchical)
        print(len(train_set))

        model = M.Siamese(separate, use_pooler, use_lscore)
        model.to(CFG.DEVICE)

        print("Training from", DATASET)
        M.train_model(model, train_set, loss_func=loss_func, shuffle=False)

        CKPT_PATH = os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "model.pth")
        if not os.path.exists(os.path.dirname(CKPT_PATH)):
            os.makedirs(os.path.dirname(CKPT_PATH))

        scorer = model.base_model
        torch.save(scorer.state_dict(), CKPT_PATH)

        del model
        torch.cuda.empty_cache()

In [4]:
def test(suffix='', separate=False, use_pooler=False, use_lscore=True):
    for DATASET in DATASETS:
        CKPT_PATH = os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "model.pth")
        scorer = M.Scorer(separate, use_pooler, use_lscore)
        scorer.load_state_dict(torch.load(CKPT_PATH, map_location=CFG.DEVICE))
        scorer.to(CFG.DEVICE)
        scorer.eval()

        E.evaluate_newsroom("human/newsroom/newsroom-human-eval.csv", os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "test_results_newsroom.tsv"), scorer)
        E.evaluate_realsumm("human/realsumm/realsumm_100.tsv", os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "test_results_realsumm.tsv"), scorer)
        E.evaluate_tac("human/tac/TAC2010_all.json", os.path.join(CFG.RESULT_ROOT, DATASET, CFG.METHOD+suffix, "test_results_tac.tsv"), scorer)

        del scorer
        torch.cuda.empty_cache()

In [5]:
train(hierarchical=False, loss_func='CrossEntropyLoss', separate=False, use_pooler=False, use_lscore=False)
test(separate=False, use_pooler=False, use_lscore=False)
# train(hierarchical=False, loss_func='MarginRankingLoss', suffix='_NonHier', separate=True)
# test(suffix='_NonHier', separate=True)

Hierarchichal False
56847
Training from billsum
CrossEntropyLoss 0.0 False


  6%|███████████▍                                                                                                                                                                           | 999/16000 [05:06<1:16:28,  3.27it/s]

Iteration 1000, Loss 377.38579911366105


 12%|██████████████████████▋                                                                                                                                                               | 1999/16000 [10:15<1:11:40,  3.26it/s]

Iteration 2000, Loss 286.1963318718481


...

 94%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋           | 14999/16000 [1:17:40<05:12,  3.20it/s]

Iteration 15000, Loss 204.6059541458817


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 15999/16000 [1:22:52<00:00,  3.23it/s]

Iteration 16000, Loss 196.55243805553528


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16000/16000 [1:22:53<00:00,  3.22it/s]
